## Building MaterialX via Python

The following is a simple example of building MaterialX from within Python.
It assumes each step will be run once for cloning, and the build step done as many times as needed.
A test of the build is done by running a MaterialX executable.

Note that this is example is written for a Windows build. Suitable build setup can be done
for Linux and Mac as well.

### Setup

Use OS and subprocess to run commands. Cache the root directory so we can return to it as needed

In [2]:
import os
import subprocess
import sys

rootdir = os.getcwd()
print('Root folder: ', rootdir)

Root folder:  d:\Work\materialx\buildMaterialX


In [3]:
def printLog(logfile):
    f = open(logfile, 'r')
    print(f.read())
    f.close()

### Clone the Repo

This will clone the ASWF repo into a subfolder called `MaterialX`:

In [59]:


print('Cloning MaterialX...')
os.chdir(rootdir)
result = os.system('git clone https://github.com/AcademySoftwareFoundation/MaterialX.git > clone.log 2>&1')
print('Finished clone: %s' % ('Success' if result == 0 else 'Failed'))
if result != 0:
    printLog('clone.log')

Cloning MaterialX...
Finished clone: Failed
fatal: destination path 'MaterialX' already exists and is not an empty directory.



# Get Submodules

Go to the MaterialX folder and get the submodules

In [60]:
os.chdir(rootdir)
os.chdir('MaterialX')

print('Get submodules...')
result = os.system('git submodule update --init --recursive > submodule.log 2>&1')
print('Finished getting submodules:', result)
if result != 0:
    printLog('submodule.log')

Get submodules...


Finished getting submodules: 0


### Create a "build" folder

In [14]:
os.chdir(rootdir)
os.chdir('MaterialX')
if not os.path.exists('build'):
    os.mkdir('build')

os.chdir('build')
print('Build folder is: ', os.getcwd())

Build folder is:  d:\Work\materialx\buildMaterialX\MaterialX\build


### Setup Platform / OS Build 

As required platform / OS pre-requisites should be installed. 

For example for Linux see [this page](https://code.visualstudio.com/docs/cpp/config-linux) which will be required when running a Codespace on remote host. gcc should already be installed but can be checked using by running `gcc -v`

In [16]:
# Pre-build setup for Linux
if sys.platform == 'linux':
    os.system('sudo apt-get update')
    os.system('sudo apt-get install xorg-dev mesa-utils')

### Setup Build Configuration MaterialX 

Go to the build folder and run CMake to set up build configuration.

In [17]:
os.chdir(rootdir)
os.chdir('MaterialX')
os.chdir('build')

# Build setup
buildSetupCmd = 'cmake -DCMAKE_EXPORT_COMPILE_COMMANDS=ON  -DMATERIALX_BUILD_VIEWER=ON -DMATERIALX_BUILD_GRAPH_EDITOR=ON -DMATERIALX_BUILD_PYTHON=ON -DMATERIALX_WARNINGS_AS_ERRORS=ON'

# Check the current OS for build setup options
osOptions = ''
if sys.platform == 'win32':
    osOptions = ' -G "Visual Studio 16 2019" -A "x64" -DCMAKE_BUILD_TYPE=RelWithDebInfo ' 
# else check if linux
elif sys.platform == 'linux':
    osOptions = ''
else:
    osOptions = ''

result = os.system(buildSetupCmd + osOptions + ' -S .. > buildsetup.log 2>&1')
print('Finished build setup: %s' % ('Success' if result == 0 else 'Failed'))
printLog('buildsetup.log')

Finished build setup: Success
-- Selecting Windows SDK version 10.0.19041.0 to target Windows 10.0.19045.
-- Setting namespace to 'MaterialX_v1_38_9'
-- NanoGUI v0.2.0 
-- NanoGUI: using OpenGL backend.
-- NanoGUI: building static library.
-- Using Win32 for window creation
-- NanoGUI: not building the Python plugin.
CMake Deprecation Warning at source/MaterialXGraphEditor/External/Glfw/CMakeLists.txt:1 (cmake_minimum_required):
  Compatibility with CMake < 3.5 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.


-- Using Win32 for window creation
CMake Deprecation Warning at source/PyMaterialX/External/PyBind11/CMakeLists.txt:8 (cmake_minimum_required):
  Compatibility with CMake < 3.5 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does

### Build MaterialX


In [9]:
os.chdir(rootdir)
os.chdir('MaterialX')
os.chdir('build')
result = os.system('cmake --build . --target install --config RelWithDebInfo > build.log 2>&1')
print('Finished build: %s' % ('Success' if result == 0 else 'Failed'))
printLog('build.log')

Finished build: Success
Microsoft (R) Build Engine version 16.11.2+f32259642 for .NET Framework
Copyright (C) Microsoft Corporation. All rights reserved.

  glfw_objects.vcxproj -> D:\Work\materialx\buildMaterialX\MaterialX\build\source\MaterialXView\external\NanoGUI\ext\glfw\src\glfw_objects.dir\RelWithDebInfo\glfw_objects.lib
  MaterialXCore.vcxproj -> D:\Work\materialx\buildMaterialX\MaterialX\build\lib\RelWithDebInfo\MaterialXCore.lib
  MaterialXFormat.vcxproj -> D:\Work\materialx\buildMaterialX\MaterialX\build\lib\RelWithDebInfo\MaterialXFormat.lib
  MaterialXGenShader.vcxproj -> D:\Work\materialx\buildMaterialX\MaterialX\build\lib\RelWithDebInfo\MaterialXGenShader.lib
  MaterialXGenGlsl.vcxproj -> D:\Work\materialx\buildMaterialX\MaterialX\build\lib\RelWithDebInfo\MaterialXGenGlsl.lib
  MaterialXGenMdl.vcxproj -> D:\Work\materialx\buildMaterialX\MaterialX\build\lib\RelWithDebInfo\MaterialXGenMdl.lib
  MaterialXGenMsl.vcxproj -> D:\Work\materialx\buildMaterialX\MaterialX\build\lib

### Test MaterialX

Run a few tests to check that the build was successful:
1. Run Python script to generate shader code
2. Run MaterialX Graph Editor

In [11]:
# Runr the generateshader.py script on one of the example materials
os.chdir(rootdir)
os.chdir('MaterialX')
result = os.system('python python/Scripts/generateShader.py resources/Materials/Examples/StandardSurface/standard_surface_glass_tinted.mtlx > genshader.log 2>&1')
printLog('genshader.log')

---------- Generate code for file:  resources/Materials/Examples/StandardSurface/standard_surface_glass_tinted.mtlx --------------------
- Set up CMS ...
- Set up Units ...
- Shader output path: d:\Work\materialx\buildMaterialX\MaterialX\resources\Materials\Examples\StandardSurface
-- Generate code for node: GlassTinted
--- Wrote pixel shader to: d:\Work\materialx\buildMaterialX\MaterialX\resources\Materials\Examples\StandardSurface/GlassTinted.glsl.frag
--- Wrote vertex shader to: d:\Work\materialx\buildMaterialX\MaterialX\resources\Materials\Examples\StandardSurface/GlassTinted.glsl.vert
--- Validation passed for node: GlassTinted



In [65]:
# Run the MaterialX Editor from the install location
os.chdir(rootdir)
cmd = 'MaterialX/build/installed/bin/MaterialXGraphEditor.exe'
if os.path.exists(cmd):
    result = subprocess.Popen([cmd, '-c 1'], stdout = subprocess.PIPE)
else:
    print('MaterialXGraphEditor.exe not found') 